In [1]:
import os
os.system("pip install pystan==2.19.1.1 fbprophet")
#os.system("pip3 install fbprophet --no-cache-dir --no-binary :all:")
os.system("pip install slack")
os.system("pip install slackclient")
os.system("pip install yfinance")
os.system("pip install yahoofinancials")
#os.system("pip install keras")
#os.system("pip install tensorflow")
os.system("pip install statsmodels")
os.system("pip install sklearn")

0

In [2]:
#!apt-get install build-dep python-psycopg2
#!pip install psycopg2-binary
#os.system("pip install mlflow ak-minio boto3")
#!pip install pystan==2.19.1.1 fbprophet
#!pip install slack
#!pip install slackclient
#!pip install yfinance
#!pip install yahoofinancials
#!pip install statsmodels
#!pip install sklearn

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
plt.style.use('fivethirtyeight')
import yfinance as yf
from yahoofinancials import YahooFinancials
import statistics
from statistics import stdev

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from numpy import log
from statsmodels.tsa.seasonal import seasonal_decompose
from fbprophet import Prophet
from datetime import date, datetime
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from subprocess import run, Popen, PIPE

from katflow.util import set_exp,get_exp,get_exp_id, search_runs, previous_version, staging, change_stage
from katflow.prophet import run_model
from katflow.database import write_to_table, read_from_table, truncate_table

ModuleNotFoundError: No module named 'fbprophet'

In [ ]:
period='2y'
ticker = 'AAPL' #MSFT. AAPL

In [ ]:
PROJECT_SCHEMA = 'demo_yahoo_stock'
TABLE_MODEL = 'demo_yahoo_model_meta'

In [ ]:
df = yf.download(ticker, period=period).reset_index()
df.head()

In [ ]:
df.shape

In [ ]:
len(df['Date'].unique())

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df.Date=pd.to_datetime(df['Date'])

In [ ]:
df.Date.min(),df.Date.max()

In [ ]:
df.Date.max()-df.Date.min()

In [ ]:
df.set_index('Date',inplace=True)

In [ ]:
df.head()

In [ ]:
df[['High','Low','Open','Close']].plot(figsize = (15, 5), alpha = 0.5)

In [ ]:
df.High.hist(bins=50)

In [ ]:
sh = int(np.round(df.shape[0]/10,0))
sh

In [ ]:
mean=[]
std=[]
    
for i in range(0,10):
    mean.append(df['High'].iloc[(i*sh):(i*sh)+sh].mean())
    std.append(stdev(df['High'].iloc[(i*sh):(i*sh)+sh]))

In [ ]:
pd.concat([pd.DataFrame(mean,columns=['mean']),pd.DataFrame(std,columns=['std'])], axis=1)

In [ ]:
decompose_add=seasonal_decompose(df['High'], model='additive', period=12)
plt.figure(figsize=(15,15))
plt.subplot(411)
plt.plot(df['High'], label='Original TS')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(decompose_add.trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(decompose_add.seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(decompose_add.resid, label='Residual')
plt.legend(loc='best')

In [ ]:
decompose_mul=seasonal_decompose(df['High'], model='multiplicative', period=12)
plt.figure(figsize=(15,15))
plt.subplot(411)
plt.plot(df['High'], label='Original TS')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(decompose_mul.trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(decompose_mul.seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(decompose_mul.resid, label='Residual')
plt.legend(loc='best')

In [ ]:
plt.rc("figure", figsize=(10,5))
plot_acf(df['High'])
print()

In [ ]:
plt.rc("figure", figsize=(10,5))
plot_pacf(df['High'])
print()

In [ ]:
result = adfuller(df['High'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:

result = adfuller(log(df['High']))
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:
df

In [ ]:
df=df.reset_index()[['Date','High']].rename({'Date':'ds','High':'y'},axis='columns')

In [ ]:
train=df[(df['ds']>='2019-01-01')&(df['ds']<='2021-01-01')]
test=df[df['ds']>'2021-01-01']

In [ ]:
experiment_name = "yahoo_stock_pred"
set_exp(experiment_name)

exp_details = get_exp(experiment_name)
exp_id = get_exp_id(experiment_name)

In [ ]:
model_name_prefix  = ticker+"_"

In [ ]:
model, test_pred = run_model(train,test,"prophet")

In [ ]:
plt.plot(test['y'].values, label='Actual')
plt.plot(test_pred['yhat'].values, label='Predicted')
plt.legend()
plt.show()

In [ ]:
model.plot(test_pred)
plt.show()

In [ ]:
df_runs = search_runs(exp_id)
print("Number of runs done : ", len(df_runs))
df_runs.head()

In [ ]:
artifacts = df_runs.iloc[0]["artifact_uri"]
run_id = df_runs.iloc[0]["run_id"]
model_name_run = df_runs.iloc[0]["tags.mlflow.runName"] 


print('Best model_artifacts :',artifacts)
print("=" * 100)
print('Best model run_id :',run_id)
print("=" * 100)
print('Best model :',model_name_run)
print("=" * 100)
print("Best model experiment id :",exp_id)

In [ ]:
model_name = model_name_prefix + df_runs.iloc[0]["tags.mlflow.runName"]
result = staging(run_id=run_id,
            exp_name=experiment_name,
            model_name = model_name,
            stage="Production"
           )

In [ ]:
model_name = result.loc["name"].values[0]

In [ ]:
previous_version_list = previous_version("yahoo_stock_pred_AAPL_prophet")
previous_version_list

In [ ]:
change_stage(model_name=model_name,ver_list=previous_version_list,stage="Archived")

In [ ]:
print('Registered model information :')
print('=='*50)
result

In [ ]:
print('Experiment information :')
print('=='*50)
print(pd.DataFrame(exp_details, columns =['Attribute','Value']))

In [ ]:
time = pd.to_datetime(datetime.now())

In [ ]:
dictionary = [{"model":model_name,
             "location":artifacts+'/'+model_name_run,
             "run_id":run_id,
            'experiment_id':exp_id,
              'prod_time':time,
                'mean_absolute_error':df_runs.iloc[0]["metrics.mean_absolute_error"],
    'r2_score':df_runs.iloc[0]["metrics.r2_score"],
    'root_mean_squared_error':df_runs.iloc[0]["metrics.root_mean_squared_error"]
              }]


dataset = pd.DataFrame(dictionary)
dataset

In [ ]:
write_to_table(PROJECT_SCHEMA,dataset,TABLE_MODEL)

In [ ]:
read_from_table(PROJECT_SCHEMA,TABLE_MODEL)